# Intro to Ray Core

This notebook introduces Ray Core, the core building block of Ray.

<div class="alert alert-block alert-info">

<b> Here is the roadmap for this notebook </b>

<ul>
    <li><b> Part 1:</b> Ray Core overview </li>
    <li><b> Part 2:</b> @ray.remote and ray.get </li>
    <li><b> Part 3:</b> Tasks can launch other tasks </li>
    <li><b> Part 4:</b> Ray Actors </li>
</ul>
</div>



## Imports

In [ ]:
import ray

## Ray Core overview

Ray Core is about:
* distributing computation across many cores, nodes, or devices (e.g., accelerators)
* scheduling *arbitrary task graphs*
    * any code you can write, you can distribute, scale, and accelerate with Ray Core
* manage the overhead
    * at scale, distributed computation introduces growing "frictions" -- data movement, scheduling costs, etc. -- which make the problem harder
    * Ray Core addresses these issues as first-order concerns in its design (e.g., via a distributed scheduler)
 
(And, of course, for common technical use cases, libraries and other components provide simple dev ex and are built on top of Ray Core)

## `@ray.remote` and `ray.get`

Here is a diagram which shows the relationship between Python code and Ray tasks.

<img src="https://technical-training-assets.s3.us-west-2.amazonaws.com/Ray_Core/python_to_ray_task_map.png" width="80%" >

Define a Python function and decorate it so that Ray can schedule it

In [ ]:
@ray.remote(num_cpus=2)
def f(a, b):
    return a + b

Tell Ray to schedule the function

In [ ]:
f.remote(1, 2)

`ObjectRef` is a handle to a task result. We get an ObjectRef immediately because we don't know
* when the task will run
* whether it will succeed
* whether we really need or want the result locally
    * consider a very large result which we may need for other work but which we don't need to inspect

In [ ]:
ref = f.remote(1, 2)

If we want to wait (block) and retrieve the corresponding object, we can use `ray.get`

In [ ]:
ray.get(ref)

<div class="alert alert-block alert-info">
    
__Activity: define and invoke a Ray task__

Define a remote function `sqrt_add` that accepts two arguments:
- computes the square-root of the first
- adds the second
- returns the result

Invoke it as a remote task with 2 different sets of parameters and collect the results

```python
# Hint: define the below as a remote function
def sqrt_add(a, b):
    ... 

# Hint: invoke it as a remote task and collect the results
```


</div>

In [ ]:
# Write your solution here

# Solution
<div class="alert alert-block alert-info">

<details>

<summary> Click to see solution </summary>

```python
import math

@ray.remote
def sqrt_add(a, b):
    return math.sqrt(a) + b

ray.get(sqrt_add.remote(2, 3)), ray.get(sqrt_add.remote(5, 4))
```

</details>

</div>


## Tasks can launch other tasks

In that example, we organized or arranged the flow of tasks from our original process -- the Python kernel behind this notebook.

Ray __does not__ require that all of your tasks and their dependencies by arranged from one "driver" process.

Consider:

In [ ]:
@ray.remote
def square(x):
    return x * x

@ray.remote
def sum_of_squares(arr):
    return sum(ray.get([square.remote(val) for val in arr]))

In [ ]:
ray.get(sum_of_squares.remote([3,4,5]))

In that example, 
* our (local) process asked Ray to schedule one task -- a call to `sum_of_squares` -- which that started running somewhere in our cluster;
* within that task, additional code requested multiple additional tasks to be scheduled -- the call to `square` for each item in the list -- which were then scheduled in other locations;
* and when those latter tasks were complete, the our original task computed the sum and completed.

This ability for tasks to schedule other tasks using uniform semantics makes Ray particularly powerful and flexible.

## Ray Actors

Actors are Python class instances which can run for a long time in the cluster, which can maintain state, and which can send messages to/from other code.

Let's look at an example of an actor which maintains a running balance.

In [ ]:
@ray.remote
class Accounting:
    def __init__(self):
        self.total = 0
    
    def add(self, amount):
        self.total += amount
        
    def remove(self, amount):
        self.total -= amount
        
    def total(self):
        return self.total

<div class="alert alert-block alert-warning">

<b>Note:</b> The most common use case for actors is with state that is not mutated but is large enough that we may want to load it only once and ensure we can route calls to it over time, such as a large AI model.

</div>

Define an actor with the `@ray.remote` decorator and then use `<class_name>.remote()` ask Ray to construct and instance of this actor somewhere in the cluster.

We get an actor handle which we can use to communicate with that actor, pass to other code, tasks, or actors, etc.

In [ ]:
acc = Accounting.remote()

We can send a message to an actor -- with RPC semantics -- by using `<handle>.<method_name>.remote()`

In [ ]:
acc.total.remote()

Not surprisingly, we get an object ref back

In [ ]:
ray.get(acc.total.remote())

We can mutate the state inside this actor instance

In [ ]:
acc.add.remote(100)

In [ ]:
acc.remove.remote(10)

In [ ]:
ray.get(acc.total.remote())

<div class="alert alert-block alert-info">

__Activity: linear model inference__

* Create an actor which applies a model to convert Celsius temperatures to Fahrenheit
* The constructor should take model weights (w1 and w0) and store them as instance state
* A convert method should take a scalar, multiply it by w1 then add w0 (weights retrieved from instance state) and then return the result

```python

# Hint: define the below as a remote actor
class LinearModel:
    def __init__(self, w0, w1):
        # Hint: store the weights

    def convert(self, celsius):
        # Hint: convert the celsius temperature to Fahrenheit

# Hint: create an instance of the LinearModel actor

# Hint: convert 100 Celsius to Fahrenheit
```

</div>

In [ ]:
# Write your solution here

<div class="alert alert-block alert-info">

<details>

<summary> Click to see solution </summary>

```python
@ray.remote
class LinearModel:
    def __init__(self, w0, w1):
        self.w0 = w0
        self.w1 = w1

    def convert(self, celsius):
        return self.w1 * celsius + self.w0

model = LinearModel.remote(w1=9/5, w0=32)
ray.get(model.convert.remote(100))
```

</details>
</div>
